In [24]:
import pandas as pd
from dowhy import CausalModel


In [25]:
data = pd.read_csv("./data/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

In [26]:
data["treatment"] = 0
data["treatment"] = data["treatment"].where(data["segment"] == "Mens E-Mail", 1)
data = data.drop(columns="segment")

In [27]:
data = data.drop(columns="mens")

In [28]:
# 疑似非RCTデータ作成

condition = (data["history"] > 300) & (data["recency"] < 6) & (data["channel"] == "Multichannel") 

not_rct_data = data.drop(data.loc[condition & (data["treatment"] == 1), :].index)
not_rct_data = data.drop(data.loc[~condition & (data["treatment"] == 0), :].index)

In [29]:
import numpy as np
struct_data = not_rct_data.copy()

non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)
print(non_numeric_columns)

['history_segment', 'zip_code', 'channel']


In [30]:

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

struct_data.head(5)

,recency,history_segment,history,womens,zip_code,newbie,channel,visit,conversion,spend,treatment
0,10,1,142.44,0,1,0,1,0,0,0.0,1
1,6,2,329.08,1,0,1,2,0,0,0.0,1
2,7,1,180.65,1,1,1,2,0,0,0.0,1
4,2,0,45.34,0,2,0,2,0,0,0.0,1
5,6,1,134.83,1,1,0,1,1,0,0.0,1


In [37]:
G = nx.DiGraph()  # 有向グラフ (Directed Graph)

# 辺の追加 (頂点も必要に応じて追加されます)
G.add_edges_from([("treatment", "visit"), ("newbie", "visit"), ("women", "visit"), ("visit", "conversation"), ("channel", "conversation"),("conversation", "spend"), ("history", "spend")])

causal_diagram = "".join(nx.generate_gml(G))

nx.nx_agraph.view_pygraphviz(G, prog='fdp')  # pygraphvizが必要

('/tmp/tmpdgkbqzae.png',
 <AGraph <Swig Object of type 'Agraph_t *' at 0x7f730f3f4a20>>)

In [38]:
model=CausalModel(
        data = struct_data,
        treatment="treatment",
        outcome="spend",
        graph=causal_diagram
        )

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['treatment'] on outcome ['spend']
